
# 🧬 NanoDockPredictor - Full Version (Google Colab)
### Created by Jaafar (2025)

✅ Smart Docking of Proteins and Ligands  
✅ AutoDock Vina + OpenBabel + Gaussian Input Generator

---

⚠️ Notes:

- This notebook is the **full version** of NanoDockPredictor.
- It runs fully in Google Colab (free).
- No need to install anything on your local machine.

---

Let's start 🚀


In [ ]:

# ✅ Install AutoDock Vina and OpenBabel
!apt-get install -y autodock-vina
!apt-get install -y openbabel
!pip install -q requests


In [ ]:

# ✅ Import libraries
import requests
import os
import subprocess
import re
import shutil


In [ ]:

# ✅ Create work directory
work_dir = "/content/nanoDocking_run"
os.makedirs(work_dir, exist_ok=True)


In [ ]:

# ✅ Download PDB and Ligand functions
def download_pdb(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_path = os.path.join(work_dir, f"{pdb_id}.pdb")
    with open(pdb_path, "w") as f:
        f.write(response.text)
    return pdb_path

def download_ligand(ligand_name):
    cid_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{ligand_name}/cids/TXT"
    cid_response = requests.get(cid_url)
    cid = cid_response.text.strip()
    sdf_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF"
    sdf_response = requests.get(sdf_url)
    sdf_path = os.path.join(work_dir, f"{ligand_name}_{cid}.sdf")
    with open(sdf_path, "w") as f:
        f.write(sdf_response.text)
    pdb_path = sdf_path.replace(".sdf", ".pdb")
    subprocess.run(f"obabel {sdf_path} -O {pdb_path} --addhydrogens", shell=True, check=True)
    return pdb_path


In [ ]:

# Example:
pdb_id = "1A3N"  # change as needed
ligand_name = "Letrozole"  # change as needed

receptor_pdb = download_pdb(pdb_id)
ligand_pdb = download_ligand(ligand_name)

print("✅ Files downloaded.")


In [ ]:

receptor_pdbqt = receptor_pdb.replace(".pdb", ".pdbqt")
ligand_pdbqt = ligand_pdb.replace(".pdb", ".pdbqt")

subprocess.run(f"obabel {receptor_pdb} -O {receptor_pdbqt} --addhydrogens", shell=True, check=True)
subprocess.run(f"obabel {ligand_pdb} -O {ligand_pdbqt} --addhydrogens", shell=True, check=True)

print("✅ Files converted to PDBQT.")


In [ ]:

vina_command = f'''
vina --receptor {receptor_pdbqt}      --ligand {ligand_pdbqt}      --center_x 0 --center_y 0 --center_z 0      --size_x 20 --size_y 20 --size_z 20      --out {work_dir}/result.pdbqt      --log {work_dir}/log.txt      --exhaustiveness 8
'''

subprocess.run(vina_command, shell=True, check=True)

print("✅ Docking completed.")


In [ ]:

with open(f"{work_dir}/log.txt", "r") as f:
    log_text = f.read()

affinities = re.findall(r"\d+\s+(-\d+\.\d+)", log_text)
print(f"📈 Binding Affinities (kcal/mol): {affinities}")


In [ ]:

ligand_name_clean = ligand_name if ligand_name.strip() != "" else "Unknown Ligand"

gaussian_text = "%mem=200MW\n"
gaussian_text += "%nprocshared=4\n"
gaussian_text += "%chk=temp.chk\n"
gaussian_text += "# opt freq=raman b3lyp/6-311g(d,p) guess=save empiricaldispersion=gd3bj\n"
gaussian_text += "scf=xqc\n\n"
gaussian_text += f"Receptor with Ligand: {ligand_name_clean}\n"

gaussian_path = os.path.join(work_dir, "gaussian_input.txt")
with open(gaussian_path, "w") as f:
    f.write(gaussian_text)

print(f"✅ Gaussian Input generated: {gaussian_path}")
